In [7]:
###prerequisite :
##download ffmpeg
#add it to paath by cmd admin :
#setx /M PATH "%PATH%;C:\Users\thiba\OneDrive - Ecole de l'air\USAFA IA\Vcode\Aero_voice_rec\ffmpeg\bin"

#download https://visualstudio.microsoft.com/fr/thank-you-downloading-visual-studio/?sku=Community&channel=Release&version=VS2022&source=VSLandingPage&cid=2030&passive=false
#for the use of LM


import gradio as gr
from datasets import load_dataset, load_metric, Audio
import tabulate
import torch
from transformers import AutoModelForCTC, Wav2Vec2Processor, Wav2Vec2ProcessorWithLM
import torchaudio.functional as F
import torchaudio

In [16]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipes = {
    "Jzuluaga/wav2vec2-large-960h-lv60-self-en-atc-atcosim": pipeline("automatic-speech-recognition", model="Jzuluaga/wav2vec2-large-960h-lv60-self-en-atc-atcosim"),
    "facebook/wav2vec2-base-960h":                           pipeline("automatic-speech-recognition", model="facebook/wav2vec2-base-960h"),
    "scy0208/whisper-aviation-base":                         pipeline("automatic-speech-recognition", model="scy0208/whisper-aviation-base"),
    "billodal/whisper-small-atc":                            pipeline("automatic-speech-recognition", model="billodal/whisper-small-atc", return_timestamps=False)  
}

Some weights of the model checkpoint at Jzuluaga/wav2vec2-large-960h-lv60-self-en-atc-atcosim were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at Jzuluaga/wav2vec2-large-960h-lv60-self-en-atc-atcosim and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weig

In [19]:
pipe_summerize = pipeline("summarization", model="facebook/bart-large-cnn")

c:\Users\thiba\OneDrive - Ecole de l'air\USAFA IA\Vcode\.env\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\thiba\.cache\huggingface\hub\models--facebook--bart-large-cnn. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [54]:
def summarize_me(text_sent):
    result = pipe_summerize(text_sent, max_length=50, min_length=10, do_sample=False)[0]['summary_text']
    print(result)
    return result

def transcribe_audio(audio_file_path):
    transcriptions = {}

    for name, pipe in pipes.items():
        try:
            # Assurez-vous d'accéder au résultat de transcription correctement
            full_text  = pipe(audio_file_path)['text']
            summerized = summarize_me('Here\'s the transcript by an AI model of an aeronotical speech : ' + full_text)

            transcriptions[name] = full_text
            transcriptions["Summerized " + name] = summerized

        except Exception as e:
            transcriptions[name] = f"Erreur de transcription: {str(e)}"
            transcriptions["Summarized " + name] = f"Erreur de résumé: {str(e)}"
        
    return [transcriptions[key] for key in transcriptions.keys()]

import gradio as gr

# Création des sorties en tant que liste de Textbox, chaque Textbox étiquetée par le nom du modèle
output_components = []
for name in pipes.keys():
    output_components.append(gr.Textbox(label=f"Transcription by {name}"))
    output_components.append(gr.Textbox(label=f"Summarized (BERT) of {name}"))

# Configuration de l'interface Gradio
iface = gr.Interface(
    fn=transcribe_audio,
    inputs=gr.Audio(label="Upload MP3 File", type="filepath"),
    outputs=output_components,  # Passer la liste de Textbox au lieu d'un dictionnaire
    title="Audio Transcription Service, Specialized in Aeronautical Speech",
    description="Upload an MP3 file to get its transcription using various fine-tuned models."
)

iface.launch()

Running on local URL:  http://127.0.0.1:7878

To create a public link, set `share=True` in `launch()`.


 AI model of an aeronotical speech. Here's the transcript.
Here's the transcript by an AI model of an aeronotical speech.
 AI model of an aeronotical speech. Here's the transcript.
